In [1]:
import socket

HOST = 'localhost'
PORT = 13052

In [2]:
def receive(sock):
    data = ""
    try:
        while 1:
            b = sock.recv(1)

            if not b:
                print(b)
                break

            data += b.decode('utf8')
    except socket.timeout:
        pass
    return data

In [11]:
import threading

class StoppableThread(threading.Thread):
    '''
        Thread class with a stop() method. The thread itself has to check regularly for the stopped() condition.
    '''

    def __init__(self):
        super(StoppableThread, self).__init__()
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def is_stopped(self):
        return self._stop_event.is_set()

    
    
    
class GameLogic():
    
    def onStatusUpdate(gamefield):
        print('-> onStatusUpdate()')
        
    def onMoveRequest():
        print('-> onMoveRequest()')


class Client(StoppableThread):

    def __init__(self, host, port, gameLogic):
        super().__init__()
        self.gameLogic = gameLogic
        
        self.sock = socket.socket()
        self.sock.connect((host, port))
    
    def run(self):
        self.sock.settimeout(0.5)
        with self.sock:
            while not self.is_stopped():
                try:
                    message = receive(self.sock)
                    self.parseMessage(message)
                except socket.error:
                    pass
                except Exception as ex:
                    print(ex)
                    raise ex
    
    def parseMessage(self, message):
        print('-> parseMessage(%s)' % message)
    
    def send(self, message):
        self.sock.send(message.encode('utf8'))

    def move(self, roomId, posX, posY, directionString):

        #let hints = move.debugHints.reduce(into: "") { $0 += "<hint content=\"\($1)\" />" }
        #let mv = "<data class=\"move\" x=\"\(move.x)\" y=\"\(move.y)\" direction=\"\(move.direction)\">\(hints)</data>"
        #self.socket.send(message: "<room roomId=\"\(self.roomId!)\">\(mv)</room>")
        hintsXML = ''
        self.send('<room roomId="%s"><data class="move" x="%d" y="%d" direction="%s">%s</data></room>' % (roomId, posX, posY, directionString, hintsXML))
        
    

In [12]:
import ipywidgets as widgets
from IPython.display import display

client = None
def toggleState(value):
    global client
    
    if value.new:
        client = Client(HOST, PORT, GameLogic())
        client.start()
        value.owner.description = 'Stop Client'
    else:
        client.stop()
        value.owner.description = 'Start Client'
        
toggle = widgets.ToggleButton(description='Start Client', value=False)
#server_toggle.value = True
toggle.observe(toggleState, names='value')

display(toggle)

ToggleButton(value=False, description='Start Client')

-> parseMessage()
-> parseMessage()
-> parseMessage()
-> parseMessage()
-> parseMessage()
-> parseMessage()
-> parseMessage()
-> parseMessage()
-> parseMessage()
-> parseMessage()
-> parseMessage()
-> parseMessage()


In [14]:
client.sock.send('<protocol><join gameType="swc_2019_piranhas"/>'.encode('utf8'))
#dataFromServer = receive()
#roomId = dataFromServer.split('"')[1]
#print(dataFromServer)

-> parseMessage()


46

-> parseMessage(<protocol>
  <joined roomId="55b6a2a0-db45-4cf4-9032-e384af7551bb"/>
  <room roomId="55b6a2a0-db45-4cf4-9032-e384af7551bb">
    <data class="welcomeMessage" color="red"/>
  </room>
  <room roomId="55b6a2a0-db45-4cf4-9032-e384af7551bb">
    <data class="memento">
      <state class="sc.plugin2019.GameState" startPlayerColor="RED" currentPlayerColor="RED" turn="0">
        <red displayName="Unknown" color="RED"/>
        <blue displayName="Unknown" color="BLUE"/>
        <board>
          <fields>
            <field x="0" y="0" state="EMPTY"/>
            <field x="0" y="1" state="RED"/>
            <field x="0" y="2" state="RED"/>
            <field x="0" y="3" state="RED"/>
            <field x="0" y="4" state="RED"/>
            <field x="0" y="5" state="RED"/>
            <field x="0" y="6" state="RED"/>
            <field x="0" y="7" state="RED"/>
            <field x="0" y="8" state="RED"/>
            <field x="0" y="9" state="EMPTY"/>
          </fields>
         

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.0/Frameworks/Python.framework/Versions/3.7/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "<ipython-input-11-59cf1cf31b48>", line 50, in run
    raise ex
  File "<ipython-input-11-59cf1cf31b48>", line 44, in run
    message = receive(self.sock)
  File "<ipython-input-2-bdd779f1f32b>", line 11, in receive
    data += b.decode('utf8')
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 0: unexpected end of data



In [ ]:
print(receive())

In [ ]:
move(roomId, 0, 2, 'DOWN_RIGHT')

In [ ]:
import declxml as xml
joined_processor = xml.string('joined', attribute='roomId')
room_processor = xml.dictionary('room', [
    xml.string('data', attribute='class'),
    xml.string('data', attribute='color', required=False, default=None),
    xml.dictionary('data', [
        xml.dictionary('state', [
            
        ], required=False)
    ])
])
protocol_processor = xml.dictionary('protocol', [
    joined_processor,
    xml.string('room', attribute='roomId'),
    xml.array(room_processor)
])

root_room_processor = xml.dictionary('root', [
    xml.array(room_processor, alias='rooms')
])

xml.parse_from_string(root_room_processor, '<root>' + '\n'.join(dataFromServer.split('\n')[2:]) + '</root>')

In [ ]:
print('<root>' + '\n'.join(dataFromServer.split('\n')[2:]) + '</root>')

In [ ]:
import re

In [ ]:
re.match(r'<room([\s\S]+?)<\/room>', 'hallo<room arg="hallo">hello\n\t</room>', re.S|re.M)

In [ ]:
for i, line in enumerate(dataFromServer.split('\n')):
    print(i, line)

In [ ]:
re.match(r'<room([\s\S]+?)</room>', dataFromServer, re.S|re.M)

In [ ]:
dataFromServer

In [ ]:
for group in re.match(r'((?:.)*?(<room.+?</room>)(?:.)*?)+', dataFromServer, re.S|re.M).groups():
    print(group)